## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-05-16-50-11 +0000,nyt,Oktoberfest’s New Drip,https://www.nytimes.com/card/2025/10/03/world/...
1,2025-10-05-16-44-45 +0000,bbc,Young Tories rate Kemi Badenoch's first year a...,https://www.bbc.com/news/videos/c2375x5endlo?a...
2,2025-10-05-16-43-05 +0000,nypost,Freed American hostage Keith Siegel thanks Tru...,https://nypost.com/2025/10/05/world-news/freed...
3,2025-10-05-16-40-24 +0000,startribune,Live: Vikings lead the Browns 21-17 in the fou...,https://www.startribune.com/minnesota-vikings-...
4,2025-10-05-16-37-03 +0000,nyt,Pritzker Says Federal Agents Are Trying to Mak...,https://www.nytimes.com/2025/10/05/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2382/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
17,trump,33
58,gaza,15
168,deal,11
48,will,9
1,new,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
38,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...,116
154,2025-10-04-20-36-56 +0000,nypost,Israel agrees to ‘initial withdrawal line’ in ...,https://nypost.com/2025/10/04/world-news/israe...,91
175,2025-10-04-17-06-03 +0000,nypost,Trump warns Hamas ‘all bets will be off’ if Ga...,https://nypost.com/2025/10/04/us-news/trump-wa...,79
171,2025-10-04-17-42-17 +0000,nypost,Trump reveals Netanyahu phone call that moved ...,https://nypost.com/2025/10/04/world-news/trump...,79
35,2025-10-05-14-49-26 +0000,nyt,"Strong-Armed by Trump, Netanyahu Embraces Gaza...",https://www.nytimes.com/2025/10/05/world/middl...,77


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
38,116,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...
154,58,2025-10-04-20-36-56 +0000,nypost,Israel agrees to ‘initial withdrawal line’ in ...,https://nypost.com/2025/10/04/world-news/israe...
134,41,2025-10-04-23-05-09 +0000,nypost,Youngkin calls for embattled Dem AG hopeful Ja...,https://nypost.com/2025/10/04/us-news/virginia...
133,32,2025-10-04-23-14-00 +0000,wsj,Defense Secretary Pete Hegseth’s emphasis on p...,https://www.wsj.com/politics/national-security...
44,26,2025-10-05-14-17-52 +0000,wapo,Russian air attack across Ukraine damages ener...,https://www.washingtonpost.com/world/2025/10/0...
13,25,2025-10-05-16-17-21 +0000,startribune,Neal: War of words between Napheesa Collier an...,https://www.startribune.com/napheesa-collier-c...
160,24,2025-10-04-19-34-28 +0000,cbc,Anti-government protesters in Georgia attempt ...,https://www.cbc.ca/news/world/georgia-protest-...
81,23,2025-10-05-09-30-00 +0000,wsj,"As Laura Loomer, an influential loyalist to Pr...",https://www.wsj.com/politics/policy/laura-loom...
21,22,2025-10-05-15-55-22 +0000,nypost,Jordan Peterson was ‘near death’ after he was ...,https://nypost.com/2025/10/05/us-news/jordan-p...
145,22,2025-10-04-21-15-20 +0000,cbc,Canada needs co-ordinated action plan to comba...,https://www.cbc.ca/news/politics/canada-action...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
